In [1]:
import pandas as pd
import numpy as np
import csv
import json
import re
import os
from tqdm import tqdm

#Sentiment Analysis Packages
import string
import nltk # preprocessing text
from wordcloud import WordCloud, STOPWORDS 
pd.options.mode.chained_assignment = None

# Create Directories

In [2]:
#  Create directories
#  Path of current working directory
current_wd = os.getcwd()
try:
    os.mkdir('{}/post_processed_data'.format(current_wd))
except:
    pass


## Data Transformation

### Business

In [3]:
def load_file(path):
    '''
    Load JSON file onto memory
    '''
    with open(path) as f:
        data = [json.loads(line) for i, line in enumerate(f)]
    df_original = pd.json_normalize(data)
    return df_original

fp = ('/mnt/data/public/yelp/dataset/yelp_dataset/'
      'yelp_academic_dataset_business.json')
df = load_file(fp)


In [4]:
#  Obtain size of dataframe
df_final = df[df['categories'].notna()]
dropped_rows = df.shape[0] - df_final.shape[0]

print(f'Original DataFrame: {df.shape}\n'
      f'Final DataFrame shape: {df_final.shape}\n'
      f'Rows dropped: {dropped_rows} '
      f'({(dropped_rows)/df.shape[0]*100:.2f})%')


Original DataFrame: (209393, 60)
Final DataFrame shape: (208869, 60)
Rows dropped: 524 (0.25)%


#### Classify each row into general category

- Obtain the item in the categories with the highest number of appearances among the list of categories.

#### Common Transformation Functions

In [5]:
def extract_dict(d, cols):
    '''
    Expand Columns containing dictionaries into additional DataFrame
    columns.
    '''
    import numpy as npb
    if isinstance(d, str):
        if d != 'None':
            dct = eval(d)
            vals = [dct.get(c, False) for c in cols]
            dct_keys = list(dct.keys())
            if not set(dct_keys).issubset(set(cols)):
                print('WARNING! WARNING! Unaccounted Dictionary Data')
            return vals
        elif d == 'None':
            return [False]*len(cols)
    elif isinstance(d, float):
        return [False]*len(cols)
    else:
        print('WARNING! WARNING! Unaccounted data')
        
        
def check_if_time(date_text):
    '''
    Check if input string is in a datetime format.
    '''
    import datetime
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d %H:%M:%S')
        return True
    except:
        return False
    
    
def classify_column(c, df):
    '''
    Classify column with respect to input data. Note that data
    familiarization should be done before running function as 
    function utilizes only the first string or boolean value of an itterable.
    '''
    data = ';'.join(map(str, set([d for d in df[c] if (isinstance(d, str)
                                                       | isinstance(d, bool))]
                                                       )))
    if ':' in data:
        sample = data.split(';')[0]
        if ',' in sample:
            sample = data.split(', ')[0]

        if check_if_time(sample):
            return 'datetime'
        else:
            return 'dictionary'
    else:
        if len(data) == 0:
            return 'Nans/Integers'
        elif (('True' in data) or ('False' in data)):
            return 'boolean'
        else:
            if "u\'" in data:
                return 'unicode_string'
            else:
                return 'string'

### Web Scraping Yelp Categories

Website that was scraped : https://blog.yelp.com/2018/01/yelp_category_list

In [6]:
import re
from bs4 import BeautifulSoup

with open("Yelp_Categories.html", encoding="utf-8") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# Finds the main content of the page which limits our string to
# the category list and their corresponding subcategories.
category_string = soup.find_all('div',
                                attrs={"class": "entry-content",
                                       "itemprop": "articleBody"})[0]

# Limits our string to a tuple where the first group would contain
# the general category and the second group would contain the multiple lines
# of subcategories.
yelp_categs = re.findall(r'<h2>(.*)</h2>([\S\s]*?)(?=<h2>|</div>)',
                         str(category_string))[1:]


# Loop that generates two dictionaries for mapping general categories
# to their respective subcategories and vice-versa.
general_to_sub = {}
sub_to_general = {}
for categ in yelp_categs:
    general_class = re.findall(r'(?:<span.*?>)(.*?)(?:</span>)',
                               str(categ[0]))
    general_class = [c.replace('&amp;', 'and') for c in general_class]
    general_class = [c.replace('&', 'and') for c in general_class]
    subclasses = re.findall(r'<li.*?>(?:<span.*?>)?' +
                            '(.*?)(?:</span>)?(?:</li>|\s<ul>)',
                            str(categ[1]))
    subclasses = [c.replace('&amp;', 'and') for c in subclasses]
    subclasses = [c.replace('&', 'and') for c in subclasses]
    subclasses.extend(general_class)
    general_to_sub[general_class[0]] = subclasses
    sub_dict = {sub: general_class[0] for sub in subclasses}
    sub_to_general.update(sub_dict)

In [7]:
# Identifying general category of each row with respect to the most frequent
# general category that the businesses are aligned with.
df_final['general_category'] = df_final.apply(lambda x: x.categories.
                                              replace('&', 'and'), axis=1)
keywords = set(list(sub_to_general.keys()))
generalized_categs = [[sub_to_general.get(k) for
                       k in keywords if k in row] for
                      row in df_final.general_category]
df_final['categories'] = [sorted(list(set([k for k in keywords if k in row])))
                          for row in df_final.general_category]
df_final['general_category'] = [max(gen_categ, key=gen_categ.count)
                                for gen_categ in generalized_categs]

In [8]:
# Drop N/A columns for brevity and lesser memory load.
df_final.drop([c for c in df_final.columns if 'N/A' in c],
              inplace=True,
              axis=1)

#### Standardize values in the different attributes

###### Extract Dictionary Values From Columns

In [9]:
df_final_column_type = pd.DataFrame([(c, classify_column(c, df_final))
                                     for c in df_final.columns],
                                    columns=['column_name', 'column_type'])
attr_dict_cols = df_final_column_type[(df_final_column_type.
                                       column_type == 'dictionary') &
                                      (df_final_column_type.
                                       column_name.str.contains
                                       ('attributes'))]

In [10]:
# Extract dictionary strings from columns
dict_cols = [c for c in attr_dict_cols.column_name
             if classify_column(c, df_final) == 'dictionary']
new_columns = []
for d in tqdm(dict_cols):
    unique_entries = set([(x, len(x)) for i, x
                          in enumerate(set(df_final[d]))
                          if isinstance(x, str)])
    dct = eval(sorted(unique_entries, key=lambda x: (-x[1], x[0]))[0][0])
    cols = list(dct.keys())
    col_names = [f'{d}.{c}' for c in cols]
    new_columns.extend(col_names)
    df_final[col_names] = df_final.apply(lambda x: extract_dict(x[d], cols),
                                         axis=1,
                                         result_type='expand')
    df_final.drop(d, inplace=True, axis=1)
df_final.head()

100%|██████████| 7/7 [01:52<00:00, 16.08s/it]


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.HairSpecializesIn.straightperms,attributes.HairSpecializesIn.coloring,attributes.HairSpecializesIn.extensions,attributes.HairSpecializesIn.africanamerican,attributes.HairSpecializesIn.curly,attributes.HairSpecializesIn.kids,attributes.HairSpecializesIn.perms,attributes.HairSpecializesIn.asian
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,...,False,False,False,False,False,False,False,False,False,False
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,...,False,False,False,False,False,False,False,False,False,False
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,...,False,False,False,False,False,False,False,False,False,False
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,...,False,False,False,False,False,False,False,False,False,False
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,...,False,False,False,False,False,False,False,False,False,False


###### Obtain attributes that contain more than 30% non-null entries. 

This is done initially since we will be creating dummy variables in the next few steps which may affect the total number of NA values within our columns. 

In [11]:
# Identifying the attributes
attr_cols = [col for col in df_final.columns if 'attr' in col]
df_attr_counts = df_final[attr_cols].isna().sum()

attr_30 = df_attr_counts[df_attr_counts < 208869*.70]
attr_30

attributes.BusinessAcceptsCreditCards            86632
attributes.BikeParking                          119104
attributes.GoodForKids                          140334
attributes.RestaurantsPriceRange2                97581
attributes.WiFi                                 143538
attributes.RestaurantsTakeOut                   142568
attributes.BusinessParking.garage                  612
attributes.BusinessParking.street                  784
attributes.BusinessParking.validated               593
attributes.BusinessParking.lot                     669
attributes.BusinessParking.valet                     0
attributes.Ambience.hipster                       1414
attributes.Ambience.romantic                      1002
attributes.Ambience.divey                         1028
attributes.Ambience.intimate                      1278
attributes.Ambience.trendy                        1774
attributes.Ambience.upscale                        873
attributes.Ambience.classy                        1228
attributes

######  Dealing with columns containing unicode strings

In [12]:
string_cols = [c for c in df_final.columns if classify_column(
    c, df_final) == 'unicode_string']
for s in string_cols:
    df_final[s] = df_final.apply(lambda x: eval(
        x[s]) if isinstance(x[s], str) else x[s], axis=1)

######  Fixing Attribute None Values

In [13]:
attributes = [a for a in df_final.columns if 'attributes.' in a]
df_final_column_type = pd.DataFrame([(c, classify_column(c, df_final))
                                     for c in attributes],
                                    columns=['column_name', 'column_type'])
#  Displaying columns and column type
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None):
    display(
        df_final_column_type[df_final_column_type['column_type'] == 'string'])

,column_name,column_type
4,attributes.RestaurantsPriceRange2,string
6,attributes.WiFi,string
7,attributes.RestaurantsAttire,string
9,attributes.NoiseLevel,string
13,attributes.Alcohol,string
26,attributes.Smoking,string
28,attributes.BYOBCorkage,string
29,attributes.AgesAllowed,string


######  Convert values to either `True`, `False` or `N/A` for boolean columns and expand string colums to dummy columns.

In [14]:
for attr in attributes:
    col_type = classify_column(attr, df_final)
    if col_type == 'boolean':
        df_final[attr] = df_final[attr].replace('True', True)
        df_final[attr] = df_final[attr].replace('False', False)
        df_final[attr] = df_final[attr].replace('None', 'N/A')
        df_final[attr] = df_final[attr].replace(np.nan, 'N/A')
        vals = list(set(df_final[attr]))
    elif col_type == 'string':
        df_final[attr] = df_final[attr].replace('no', 'N/A')
        df_final[attr] = df_final[attr].replace('None', 'N/A')
        df_final[attr] = df_final[attr].replace('none', 'N/A')
        df_final[attr] = df_final[attr].replace(np.nan, 'N/A')

        vals = list(set(df_final[attr]))
        print(attr, vals)
        for v in vals:
            df_final[attr + f'.{v.title()}'] = df_final[
                attr]\
                .replace(v, True)
            df_final[attr + f'.{v.title()}'] = np.where(
                df_final[
                    attr
                    + f'.{v.title()}']
                == True, True, False)
    else:
        df_final[attr] = df_final[attr]

attributes.RestaurantsPriceRange2 ['2', '4', 'N/A', '1', '3']
attributes.WiFi ['free', 'N/A', 'paid']
attributes.RestaurantsAttire ['formal', 'N/A', 'casual', 'dressy']
attributes.NoiseLevel ['loud', 'N/A', 'quiet', 'average', 'very_loud']
attributes.Alcohol ['full_bar', 'N/A', 'beer_and_wine']
attributes.Smoking ['N/A', 'outdoor', 'yes']
attributes.BYOBCorkage ['yes_corkage', 'yes_free', 'N/A']
attributes.AgesAllowed ['19plus', '18plus', 'N/A', 'allages', '21plus']


In [15]:
# Based on the decision of the team, we decided to focus on these columns
col_names = ['business_id', 'name', 'state', 'latitude', 'longitude', 'stars',
             'review_count', 'is_open', 'categories', 'general_category',
             'attributes.BusinessAcceptsCreditCards', 'attributes.BikeParking',
             'attributes.GoodForKids', 'attributes.RestaurantsPriceRange2',
             'attributes.RestaurantsTakeOut',
             'attributes.WheelchairAccessible',
             'attributes.WiFi', 'attributes.Smoking',
             'attributes.ByAppointmentOnly',
             'attributes.BusinessParking.garage',
             'attributes.BusinessParking.street',
             'attributes.BusinessParking.validated',
             'attributes.BusinessParking.lot',
             'attributes.BusinessParking.valet']
df_final = df_final[col_names]

In [16]:
df_final

,business_id,name,state,latitude,longitude,stars,review_count,is_open,categories,general_category,...,attributes.RestaurantsTakeOut,attributes.WheelchairAccessible,attributes.WiFi,attributes.Smoking,attributes.ByAppointmentOnly,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,NC,35.462724,-80.852612,3.5,36,1,"[Active Life, Gun/Rifle Ranges, Guns and Ammo,...",Shopping,...,N/A,N/A,N/A,N/A,False,False,False,False,True,False
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",AZ,33.569404,-111.890264,5.0,4,1,"[Active Life, Fitness and Instruction, Health ...",Active Life,...,N/A,N/A,N/A,N/A,True,False,False,False,False,False
2,XNoUzKckATkOD1hP6vghZg,Felinus,QC,45.479984,-73.580070,5.0,5,1,"[Pet Groomers, Pet Services, Pets]",Pets,...,N/A,N/A,N/A,N/A,N/A,False,False,False,False,False
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,NV,36.219728,-115.127725,2.5,3,0,"[Building Supplies, Hardware Stores, Home Serv...",Shopping,...,N/A,N/A,N/A,N/A,False,False,False,False,True,False
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,AZ,33.428065,-111.726648,4.5,26,1,"[Contractors, Electricians, Handyman, Home Ser...",Home Services,...,N/A,N/A,N/A,N/A,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,ON,43.838555,-79.559823,4.0,5,0,"[Bars, Japanese, Restaurants, Sushi Bars]",Restaurants,...,True,N/A,free,N/A,N/A,False,False,False,False,False
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,NC,35.226307,-80.893539,2.0,13,0,"[Department Stores, Fashion, Food, Grocery, Mo...",Shopping,...,N/A,N/A,N/A,N/A,N/A,False,False,False,True,False
209390,SYa2j1boLF8DcGVOYfHPcA,Five Guys,AZ,33.504062,-111.929431,3.5,97,1,"[Burgers, Fast Food, Food, Restaurants]",Restaurants,...,True,N/A,N/A,N/A,N/A,False,False,False,True,False
209391,RSSIsgO00OuWQTRoITacpA,Indian Trail Dog Training,NC,35.101965,-80.599385,5.0,3,1,"[Pet Services, Pet Training, Pets]",Pets,...,N/A,N/A,N/A,N/A,N/A,False,False,False,False,False


In [17]:
#  Save DataFrame to csv
df_final.to_csv('post_processed_data/yelp_processed_dataset.csv',
                index=False)

### Reviews

In [18]:
def preprocess(x):
    '''
    Transform strings to their lower case form and remove
    common stopwords and punctuations.
    '''
    pctn = list(string.punctuation)
    sw_pctn = set(STOPWORDS).union(pctn)
    x = re.sub('[^a-z\s]', '', x.lower())
    x = [w for w in x.split() if w not in set(sw_pctn)]
    return ' '.join(x)

#### Preparing mapping functions

In [19]:
df_final = pd.read_csv('post_processed_data/yelp_processed_dataset.csv')

In [20]:
business = df_final.copy()

# business_id to State
bid_to_state = dict(zip(business.business_id, business.state))

# business_id to Category
bid_to_categ = dict(zip(business.business_id, business.categories))

# business_id to General Category
bid_to_gen_categ = dict(zip(business.business_id, business.general_category))

In [21]:
# Getting the Top 9 States which we will be using later on.
SOIs = business.state.value_counts().keys().tolist()[:9]

In [22]:
file = '/mnt/data/public/yelp/dataset/yelp_dataset/' +\
       'yelp_academic_dataset_review.json'
chunks = pd.read_json(file, lines=True, chunksize=100000)
state_travelled = []

#  Map business categories and states to each review using
#  business_id as a reference.
for chunk in tqdm(chunks):
    review = chunk
    review['state'] = review.apply(lambda x:
                                   bid_to_state.get(x.business_id),
                                   axis=1)
    review['categories'] = review.apply(lambda x:
                                        bid_to_categ.get(x.business_id),
                                        axis=1)
    review['general_category'] = review.apply(lambda x:
                                              bid_to_gen_categ.
                                              get(x.business_id),
                                              axis=1)
    review = review[~(review.state.isna()) |
                    ~(review.categories.isna()) |
                    ~(review.general_category.isna())]
    review = review[['review_id',
                     'user_id',
                     'business_id',
                     'stars',
                     'state',
                     'categories',
                     'general_category',
                     'text']]
    for state in SOIs:
        state_business = business[business.state == state]
        top_100 = state_business.sort_values(['review_count',
                                              'stars'],
                                             ascending=False)[:100]
        top_100_ids = list(set(top_100.business_id))

        #  Parse only data that will be relevant to use-case(WordCloud)
        #  as to lessen runtime load.
        state_review = review[(review.state == state) &
                              (review.stars >= 3) &
                              (review.business_id.isin(top_100_ids))]
        state_review['text_prcs'] = state_review.text.apply(preprocess)
        if state not in state_travelled:
            state_review.to_csv(f'post_processed_data/' +
                                f'processed_review_{state}.csv',
                                index=False)
            state_travelled.append(state)
        else:
            state_review.to_csv(f'post_processed_data/' +
                                f'processed_review_{state}.csv',
                                mode='a',
                                header=False,
                                index=False)

## Data Aggregation

### Rating

In [23]:
df_final = pd.read_csv('post_processed_data/yelp_processed_dataset.csv')


def star_rate(x):
    '''
    Classify a business' star rating based on the metric that
    businesses with ratings above 3 are "Good" stores and businesses
    with ratigns below 3 are "Bad" stores. All businesses rated 3 stars
    are considered "Neutral"-rated.
    '''
    if x > 3:
        return 'Good'
    elif x == 3:
        return 'Neutral'
    else:
        return 'Bad'


df_final['rating'] = df_final['stars'].apply(star_rate)
df_final['rating'].value_counts()

Good       134197
Bad         46078
Neutral     28594
Name: rating, dtype: int64

In [24]:
#  Generate percentages with regards to the ratings we designated
#  in the previous cell.

df_rating = pd.DataFrame(df_final.groupby(
    ['state', 'general_category'])['business_id'].count())
df_rating = df_rating[df_rating['business_id'] > 100]
df_rating['pct_good'] = (df_final[df_final['rating'] == 'Good']
                         .groupby(['state', 'general_category'])
                         ['business_id'].count()
                         / df_rating['business_id'] * 100)
df_rating['pct_neutral'] = (df_final[df_final['rating'] == 'Neutral']
                            .groupby(['state', 'general_category'])
                            ['business_id'].count()
                            / df_rating['business_id'] * 100)
df_rating['pct_bad'] = (df_final[df_final['rating'] == 'Bad']
                        .groupby(['state', 'general_category'])
                        ['business_id'].count()
                        / df_rating['business_id'] * 100)
df_rating['pct_neutbad'] = df_rating['pct_neutral'] + df_rating['pct_bad']
df_rating['pct_neutgood'] = df_rating['pct_neutral'] + df_rating['pct_good']
df_rating['check'] = df_rating['pct_good'] + \
    df_rating['pct_bad'] + df_rating['pct_neutral']
df_rating.to_csv('post_processed_data/ratings.csv')

In [25]:
df_rating

business_id   pct_good  pct_neutral    pct_bad  \
state general_category                                                     
AB    Active Life                 301  65.780731    15.282392  18.936877   
      Automotive                  509  43.811395    13.555992  42.632613   
      Beauty and Spas             702  51.994302    13.817664  34.188034   
      Food                       1103  68.993654    16.047144  14.959202   
      Health and Medical          396  57.070707     9.343434  33.585859   
...                               ...        ...          ...        ...   
WI    Nightlife                   301  71.428571    17.275748  11.295681   
      Pets                        157  81.528662    12.101911   6.369427   
      Real Estate                 136  35.294118    10.294118  54.411765   
      Restaurants                1343  60.312733    18.093820  21.593448   
      Shopping                    800  63.000000    13.500000  23.500000   

                          pct_neutbad  pct_neutgood  check  
state general_category                                      
AB    Active Life           34.219269     81.063123  100.0  
      Automotive            56.188605     57.367387  100.0  
      Beauty and Spas       48.005698     65.811966  100.0  
      Food                  31.006346     85.040798  100.0  
      Health and Medical    42.929293     66.414141  100.0  
...                               ...           ...    ...  
WI    Nightlife             28.571429     88.704319  100.0  
      Pets                  18.471338     93.630573  100.0  
      Real Estate           64.705882     45.588235  100.0  
      Restaurants           39.687267     78.406552  100.0  
      Shopping              37.000000     76.500000  100.0  

[157 rows x 7 columns]

### Radar Chart

Attributes for BusinessParking have sub attributes. Combine to single attribute to represent whether an establishment has at least one of available BusinessParking options.

In [26]:
df_final.columns

Index(['business_id', 'name', 'state', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'categories', 'general_category',
       'attributes.BusinessAcceptsCreditCards', 'attributes.BikeParking',
       'attributes.GoodForKids', 'attributes.RestaurantsPriceRange2',
       'attributes.RestaurantsTakeOut', 'attributes.WheelchairAccessible',
       'attributes.WiFi', 'attributes.Smoking', 'attributes.ByAppointmentOnly',
       'attributes.BusinessParking.garage',
       'attributes.BusinessParking.street',
       'attributes.BusinessParking.validated',
       'attributes.BusinessParking.lot', 'attributes.BusinessParking.valet',
       'rating'],
      dtype='object')

Among top 7 attributes which YELP users answered (not nan), we are showing the percentage of users that confirmed the establishments have the attribute.

In [27]:
# Create a DataFrame that would store values for BusinessParking related
# data which will be aggregated onto another dataframe for our radar charts.
generic_attr = ['state', 'attributes.BusinessAcceptsCreditCards',
                'attributes.BikeParking', 'attributes.GoodForKids',
                'attributes.WiFi', 'attributes.RestaurantsTakeOut',
                'attributes.HasTV', 'attibutes.WheelchairAccessible',
                'attributes.Smoking']
park = ['attributes.BusinessParking.garage',
        'attributes.BusinessParking.street',
        'attributes.BusinessParking.validated',
        'attributes.BusinessParking.lot',
        'attributes.BusinessParking.valet']

df_park = df_final[['state']+park].copy()
df_park.replace(['N/A'], False, inplace=True)
df_park['attributes.BusinessParking'] = df_park[park].any(axis=1)

#  Generating a subDataFrame containing data that will be used to
#  create radar charts.
df_new = pd.DataFrame(df_final['state'].value_counts())
df_new['CreditCards'] = (df_final[
                         df_final['attributes.BusinessAcceptsCreditCards']
                         == True].groupby('state')
                         ['attributes.BusinessAcceptsCreditCards']
                         .count() / df_new['state'] * 100)
df_new['Parking'] = (df_park[df_park['attributes.BusinessParking']
                     == True].groupby('state')['attributes.BusinessParking']
                     .count() / df_new['state'] * 100)
df_new['Biking'] = (df_final[df_final['attributes.BikeParking'] == True]
                    .groupby('state')['attributes.BikeParking']
                    .count() / df_new['state'] * 100)
df_new['ByAppointmentOnly'] = (df_final[
    df_final['attributes.ByAppointmentOnly'] == True]
    .groupby('state')['attributes.ByAppointmentOnly']
    .count() / df_new['state'] * 100)
df_new['GoodForKids'] = (df_final[df_final['attributes.GoodForKids'] == True]
                         .groupby('state')['attributes.GoodForKids']
                         .count() / df_new['state'] * 100)
df_new['RestaurantsTakeOut'] = (df_final[
                                df_final['attributes.RestaurantsTakeOut']
                                == True].groupby('state')
                                ['attributes.RestaurantsTakeOut']
                                .count() / df_new['state'] * 100)
df_new['WiFi'] = ((df_final[df_final['attributes.WiFi'] == 'free']
                   .groupby('state')['attributes.WiFi']
                   .count() +
                  df_final[df_final['attributes.WiFi'] == 'paid']
                   .groupby('state')['attributes.WiFi']
                   .count()) / df_new['state'] * 100)
df_new.replace(['N/A'], False, inplace=True)
df_radar = (df_new[0:9].drop('state', axis=1)
                       .rename_axis('state').reset_index())
df_radar.columns = ['state', 'Credit Cards', 'Parking', 'Bike\nParking',
                    'Appointment\nOnly', 'Good \nFor Kids',
                    'Restaurant\nTake\nOut', 'WiFi']

In [28]:
df_radar.to_csv('post_processed_data/radar.csv', index = False)

In [29]:
df_radar

,state,Credit Cards,Parking,Bike\nParking,Appointment\nOnly,Good \nFor Kids,Restaurant\nTake\nOut,WiFi
0,AZ,74.479734,29.125029,31.250618,18.178820,21.335048,20.182382,16.068072
1,NV,73.700816,32.185913,29.525984,15.810804,22.626071,20.979326,16.744472
2,ON,2.142857,36.598248,39.693487,6.740558,33.535851,40.678708,19.961686
3,OH,74.316137,34.220672,31.503580,7.447525,29.471881,34.110520,19.105318
4,NC,75.763194,32.641206,32.560870,11.673464,26.745767,28.939562,19.132369
5,PA,73.942977,36.902641,32.974243,8.917868,28.130569,33.573627,17.827636
6,QC,1.870715,28.050930,38.423115,3.036239,38.285994,45.406464,23.428012
7,AB,2.343840,32.097910,34.857407,7.770465,27.606512,32.721395,19.431936
8,WI,75.802066,40.619902,43.121262,10.186696,28.022476,31.049483,20.917165


# Creating Top 5 Categories

In [30]:
# creating the df_top5_cats.csv file
df = pd.read_csv('post_processed_data/yelp_processed_dataset.csv')

top_states = ['AZ', 'NV', 'ON', 'OH', 'NC', 'PA', 'QC', 'AB', 'WI']

df_top = df[df['state'].isin(top_states)]

df_state_cat_top = pd.DataFrame(df_top.groupby(['state',
                                                'general_category'])\
                                                ['business_id'].count())
df_state_cat_top = df_state_cat_top.reset_index()
df_state_cat_top.columns = ['state', 'general_categories', 'count']

df_state_cat_top = df_state_cat_top.sort_values(by=['state', 'count'],
                                                ascending=[True, False])\
                                                .reset_index(drop=True)

df_top5_cats = df_state_cat_top.groupby('state').head(5)

def get_total(state):
    return df['state'].value_counts().get(state)

df_top5_cats['total_establishments'] = df_top5_cats['state'].apply(get_total)
df_top5_cats['% of total'] = (df_top5_cats['count']\
                              /df_top5_cats['total_establishments'])*100

df_top5_cats.to_csv('post_processed_data/df_top5_cats.csv', index=False)

In [31]:
df_top5_cats

,state,general_categories,count,total_establishments,% of total
0,AB,Restaurants,2524,8661,29.142131
1,AB,Shopping,1207,8661,13.936035
2,AB,Food,1103,8661,12.735250
3,AB,Beauty and Spas,702,8661,8.105300
4,AB,Automotive,509,8661,5.876920
22,AZ,Restaurants,9867,60642,16.270901
23,AZ,Shopping,7308,60642,12.051054
24,AZ,Health and Medical,7168,60642,11.820191
25,AZ,Home Services,6027,60642,9.938656
26,AZ,Beauty and Spas,5007,60642,8.256654
